# Étude de l’impact du VIX sur les marchés boursiers mondiaux

**Problématique :** Dans quelle mesure l’évolution du VIX permet-elle d’expliquer ou d’anticiper les variations des marchés boursiers internationaux ?

On cherche à savoir si une hausse du VIX (la “peur” sur Wall Street) provoque une baisse ou une réaction similaire sur les marchés européens et asiatiques

---

### Importation des packages

In [ ]:
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

---

### Importation des données 

In [ ]:
start_date = '2007-03-30'
end_date = pd.Timestamp.today().strftime('%Y-%m-%d')

# data for VIX and stock indices

tickers = {
    'VIX': '^VIX',
    'S&P500': '^GSPC',
    'EuroStoxx50': '^STOXX50E',
    'Nikkei225': '^N225'
}

dic = {}

for key, ticker in tickers.items():
    data = yf.download(ticker, start=start_date, end=end_date)
    dic[key] = data

data = pd.concat(dic, axis=1)

# # exchange rates to convert to USD

# fx = {
#     'EURUSD': 'EURUSD=X',  # Euro -> USD
#     'JPYUSD': 'JPYUSD=X'   # Yen -> USD
# }

# dic_fx = {}

# for key, ticker in fx.items():
#     data_fx = yf.download(ticker, start=start_date, end=end_date)
#     dic_fx[key] = data_fx['Close']

# fx_data = pd.concat(dic_fx, axis=1)

# function to convert to USD
    
print(data.tail())

--- 

### Représentation graphique

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(14, 8), sharex=True)

axs[0, 0].plot(data['VIX']['Close'], color='black')
axs[0, 0].set_title('VIX')
axs[0, 0].set_ylabel('Valeur')

axs[0, 1].plot(data['S&P500']['Close'], color='orangered')
axs[0, 1].set_title('S&P500')
axs[0, 1].set_ylabel('Valeur')

axs[1, 0].plot(data['EuroStoxx50']['Close'], color='navy')
axs[1, 0].set_title('EuroStoxx50')
axs[1, 0].set_ylabel('Valeur')

axs[1, 1].plot(data['Nikkei225']['Close'], color='yellowgreen')
axs[1, 1].set_title('Nikkei225')
axs[1, 1].set_ylabel('Valeur')

plt.tight_layout()
plt.show()